# Diet Optimization 
##### Alan Kessler, 7/3/17

"If It Fits Your Macros" is a nutrition concept that essentially says, "eat whatever you want...if it fits your macros". Macros are short for three macronutrients: protein, fat, and carbohydrates. The thought is that you can diet more simply by planning around just these three macros, and that the goal of the diet is more important than how you get there. Being a "numbers person", I have spent considerably more time trying to figure out how meet the macro thresholds than actually trying the diet. 

The issue I have had is has to do with how the formula works. It starts with looking at your daily Total Daily Energy Expenditure (TDEE). To put it simply, this is the total number of calories you burn during the day. From there, you get set a goal: either gaining or losing weight at a rate of your choice. The goal dictates your macros. The "algorithm" if you can call it that, assumes a number of calories per gram for each macronutrient. These constants are thought to be averages for nutritionally dense foods. I want to put these averages to the test.    

My diet failures make for a great optimization problem. The goal is to meet my macro targets with the minimum number of calories. My suspicion is that it will take considerably more calories than my goal to meet my macros. 

## Calorie Requirement & Macros

I used the "If It Fits Your Macros" website to make the calculations. Note that this website has quite a few ads and requires an email address to get results.

The website provided me results of:

- **Calories:** 2,088 per day
- **Protein:** 167 grams per day
- **Fat:** 77 grams per day
- **Carbohydrates:** 183 grams per day

IIFYM Macro Calculator. (n.d.). Retrieved July 03, 2017, from https://www.iifym.com/iifym-calculator/

## Food

The three foods I have selected are:

#### [Brown Rice](https://www.google.com/#q=brown+rice+nutrition) $X_1$:
- Serving size: 1 cup, cooked
- Calories: 216
- Protein: 5g
- Fat: 1.8g
- Carbohydrates: 45g

#### [Avocados](https://www.google.com/#q=avocado+nutrition) $X_2$:
- Serving size: 1 cup sliced
- Calories: 234
- Protein: 2.9g
- Fat: 21g
- Carbohydrates: 12g  

#### [Chicken Breast](https://www.google.com/#q=chicken+breast+nutrition) $X_3$:
- Serving size: 1 cup chopped
- Calories: 231
- Protein: 43g
- Fat: 5g
- Carbohydrates: 0g

The nutrition information is included with each food from quick Google searches.

## Goals - Simplified

Minimize daily calories consumed subject to:

1. Consume at least 167 grams of protein
2. Consume at least 77 grams of fat
3. Consume at least 183 grams of carbohydrates
4. Consume a non-negative amount of each food

## Goals - Represented Mathematically

Minimize: $Calories = 216X_1 + 234X_2 + 231X_3$

Subject to:

1. $5.0X_1 + 2.9X_2 + 43.0X_3 \geq 167$
2. $1.8X_1 + 21.0X_2 + 5.0X_3 \geq 77$
3. $45.0X_1 + 12.0X_2 + 0X_3 \geq 183$
4. $X_1 \geq 0, X_2 \geq 0, X_3 \geq 0$

## Simplex Method

In [1]:
"""Demonstrate Simplex Method for Optimization."""

import numpy as np
from scipy.optimize import linprog
np.set_printoptions(suppress=True, precision=4, linewidth=125)

# Original matrix
original = np.matrix(([5, 2.9, 43, 167], [1.8, 21, 5, 77],
                      [45, 12, 0, 183], [216, 234, 231, 0]))

# Transpose for dual
dual = np.transpose(original)
# Switch signs for minimize vector
dual[3] = dual[3]*-1

# Insert the slack variables
rs = np.matrix(([1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]))
step0 = np.concatenate((dual[:, :3], rs, dual[:, 3]), axis=1)
print(step0)

[[   5.     1.8   45.     1.     0.     0.     0.   216. ]
 [   2.9   21.    12.     0.     1.     0.     0.   234. ]
 [  43.     5.     0.     0.     0.     1.     0.   231. ]
 [-167.   -77.  -183.     0.     0.     0.     1.     0. ]]


In [2]:
# Step 1: Pivot is column 3, row 1
step1 = np.copy(step0)
step1[0] = step1[0]/step1[0, 2]
step1[1] = step1[1] - step1[1, 2]*step1[0]
step1[3] = step1[3] - step1[3, 2]*step1[0]
print(step1)

[[   0.1111    0.04      1.        0.0222    0.        0.        0.        4.8   ]
 [   1.5667   20.52      0.       -0.2667    1.        0.        0.      176.4   ]
 [  43.        5.        0.        0.        0.        1.        0.      231.    ]
 [-146.6667  -69.68      0.        4.0667    0.        0.        1.      878.4   ]]


In [3]:
# Step 2: Pivot is column 1, row 3
step2 = np.copy(step1)
step2[2] = step2[2]/step2[2, 0]
step2[0] = step2[0] - step2[2]*step2[0, 0]
step2[1] = step2[1] - step2[2]*step2[1, 0]
step2[3] = step2[3] - step2[2]*step2[3, 0]
print(step2)

[[   0.        0.0271    1.        0.0222    0.       -0.0026    0.        4.2031]
 [   0.       20.3378    0.       -0.2667    1.       -0.0364    0.      167.9837]
 [   1.        0.1163    0.        0.        0.        0.0233    0.        5.3721]
 [   0.      -52.6257    0.        4.0667    0.        3.4109    1.     1666.307 ]]


In [4]:
# Step 3: Pivot is column 2, row 2
step3 = np.copy(step2)
step3[1] = step3[1]/step3[1, 1]
step3[0] = step3[0]-step3[1]*step3[0, 1]
step3[2] = step3[2]-step3[1]*step3[2, 1]
step3[3] = step3[3]-step3[1]*step3[3, 1]
print(step3)

[[   0.        0.        1.        0.0226   -0.0013   -0.0025    0.        3.9794]
 [   0.        1.        0.       -0.0131    0.0492   -0.0018    0.        8.2597]
 [   1.        0.        0.        0.0015   -0.0057    0.0235    0.        4.4117]
 [   0.        0.        0.        3.3766    2.5876    3.3166    1.     2100.9781]]


In [5]:
print("The solution:")
print(f"Eat {step3[3, 3]:.4f} servings of brown rice per day.")
print(f"Eat {step3[3, 4]:.4f} servings of avocados per day.")
print(f"Eat {step3[3, 5]:.4f} servings of chicken breast per day.")
print(f"For a total of {step3[3, 7]:.4f} calories per day.")

The solution:
Eat 3.3766 servings of brown rice per day.
Eat 2.5876 servings of avocados per day.
Eat 3.3166 servings of chicken breast per day.
For a total of 2100.9781 calories per day.


## Check Results Using Python (SciPy)

In [6]:
# Vector of to minimize
z = [216, 234, 231]

# Coefficients on the left side of the matrix
A = [[-5, -2.9, -43], [-1.8, -21, -5], [-45, -12, 0]]

# Results from the right side of the matrix
b = [-167, -77, -183]

# Bounds
x1_bounds = (0, None)
x2_bounds = (0, None)
x3_bounds = (0, None)

linprog(z, A_ub=A, b_ub=b, bounds=(x1_bounds, x2_bounds, x3_bounds),
        options={"disp": True})


Optimization terminated successfully.
         Current function value: 2100.978093 
         Iterations: 3


     fun: 2100.9780925300543
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([0., 0., 0.])
  status: 0
 success: True
       x: array([3.3766, 2.5876, 3.3166])

## Conclusion

The final result of 2,101 calories is fairly close to the 2,088 of my goal. It is likely that a more varied diet would result in a number closer to the goal. However, the three foods I chose are all fairly nutritionally dense. As a result, if I were to try this diet, I would do my best to eat these types of foods but focus on the calorie number and only use the macros as a goal. Achieving both calorie and macro goals at the same time may be difficult without supplements like protein poweder. 